In [13]:
import matplotlib.pyplot as plt
import numpy as np
from functools import partial
# from pyscf import dft, scf, gto, cc
from pyscfad import dft, scf, gto, cc
from ase.io import read
import equinox as eqx
import jax

In [2]:
g297 = read('/home/awills/Documents/Research/xcquinox/scripts/script_data/haunschild_g2/g2_97.traj', ':')

In [3]:
h2 = [i for i in g297 if i.get_chemical_formula() == 'H2'][0]

In [4]:
h2

Atoms(symbols='H2', pbc=False, calculator=SinglePointCalculator(...))

In [5]:
syms = h2.get_chemical_symbols()
pos = h2.positions
atstr = '; '.join(['{} {} {} {}'.format(syms[i], pos[i][0], pos[i][1], pos[i][2]) for i in range(len(syms))])
mol = gto.Mole(atom=atstr, basis='def2-tzvpd')
mol.verbose = 4
mf = dft.RKS(mol)
mf.xc = 'pbe,pbe'
mf.diis = None
mf.max_cycle = 25
mf.diis_start_cycle = 50
mf.grids.level = 5
mf.kernel(linear_mixing_alpha=0.3)

Initialize <pyscfad.gto.mole.Mole object at 0x7f5c7c7ef970> in <pyscfad.dft.rks.RKS object at 0x7f5c7c7ee320>
An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


System: uname_result(system='Linux', node='aegis', release='5.15.0-139-generic', version='#149~20.04.1-Ubuntu SMP Wed Apr 16 08:29:56 UTC 2025', machine='x86_64')  Threads 20
Python 3.10.14 (main, Mar 21 2024, 16:24:04) [GCC 11.2.0]
numpy 1.26.4  scipy 1.11.4  h5py 3.11.0
Date: Mon Jul  7 10:22:36 2025
PySCF version 2.9.0
PySCF path  /home/awills/anaconda3/envs/pyscfad/lib/python3.10/site-packages/pyscf

[CONFIG] conf_file None
[INPUT] verbose = 4
[INPUT] num. atoms = 2
[INPUT] num. electrons = 2
[INPUT] charge = 0
[INPUT] spin (= nelec alpha-beta = 2S) = 0
[INPUT] symmetry False subgroup None
[INPUT] Mole.unit = angstrom
[INPUT] Symbol           X                Y                Z      unit          X                Y                Z       unit  Magmom
[INPUT]  1 H      0.000000000000   0.000000000000   0.371395000000 AA    0.000000000000   0.000000000000   0.701834834033 Bohr   0.0
[INPUT]  2 H      0.000000000000   0.000000000000  -0.371395000000 AA    0.000000000000   0.0000000000

/home/awills/anaconda3/envs/pyscfad/lib/python3.10/site-packages/pyscf/gto/mole.py:1293: UserWarning: Function mol.dumps drops attribute coords because it is not JSON-serializable
  warnings.warn(msg)
/home/awills/anaconda3/envs/pyscfad/lib/python3.10/site-packages/pyscf/gto/mole.py:1293: UserWarning: Function mol.dumps drops attribute exp because it is not JSON-serializable
  warnings.warn(msg)
/home/awills/anaconda3/envs/pyscfad/lib/python3.10/site-packages/pyscf/gto/mole.py:1293: UserWarning: Function mol.dumps drops attribute ctr_coeff because it is not JSON-serializable
  warnings.warn(msg)
/home/awills/anaconda3/envs/pyscfad/lib/python3.10/site-packages/pyscf/gto/mole.py:1293: UserWarning: Function mol.dumps drops attribute _enuc because it is not JSON-serializable
  warnings.warn(msg)


cycle= 1 E= -1.2392768795632  delta_E= -0.277  |g|= 4.62e-15  |ddm|= 0.466
LINEAR MIXING SPECIFIED, ALPHA =  0.3
LINEAR MIXING SPECIFIED -- Using alternate order for matrix calculations
  HOMO = -0.389849011442173  LUMO = 0.0274400575140182
cycle= 2 E= -1.18249030867976  delta_E= 0.0568  |g|= 1.47e-14  |ddm|= 0.178
LINEAR MIXING SPECIFIED, ALPHA =  0.3
LINEAR MIXING SPECIFIED -- Using alternate order for matrix calculations
  HOMO = -0.38497907633069  LUMO = 0.0290799291551159
cycle= 3 E= -1.17393479523424  delta_E= 0.00856  |g|= 4.87e-15  |ddm|= 0.108
LINEAR MIXING SPECIFIED, ALPHA =  0.3
LINEAR MIXING SPECIFIED -- Using alternate order for matrix calculations
  HOMO = -0.383069137270172  LUMO = 0.0297201898268427
cycle= 4 E= -1.17043912620963  delta_E= 0.0035  |g|= 3.82e-15  |ddm|= 0.0729
LINEAR MIXING SPECIFIED, ALPHA =  0.3
LINEAR MIXING SPECIFIED -- Using alternate order for matrix calculations
  HOMO = -0.382082357126969  LUMO = 0.0300483994201873
cycle= 5 E= -1.16857112748194  d

Array(-1.16587383, dtype=float64)

In [6]:
syms = h2.get_chemical_symbols()
pos = h2.positions
atstr = '; '.join(['{} {} {} {}'.format(syms[i], pos[i][0], pos[i][1], pos[i][2]) for i in range(len(syms))])
mol = gto.Mole(atom=atstr, basis='def2-tzvpd')
mol.verbose = 4
mf = dft.RKS(mol)
mf.xc = 'pbe,pbe'
mf.max_cycle = 25
mf.diis_start_cycle = 50
mf.grids.level = 5
mf.kernel(linear_mixing_alpha=0.3)

System: uname_result(system='Linux', node='aegis', release='5.15.0-139-generic', version='#149~20.04.1-Ubuntu SMP Wed Apr 16 08:29:56 UTC 2025', machine='x86_64')  Threads 20
Python 3.10.14 (main, Mar 21 2024, 16:24:04) [GCC 11.2.0]
numpy 1.26.4  scipy 1.11.4  h5py 3.11.0
Date: Mon Jul  7 10:22:39 2025
PySCF version 2.9.0
PySCF path  /home/awills/anaconda3/envs/pyscfad/lib/python3.10/site-packages/pyscf

[CONFIG] conf_file None
[INPUT] verbose = 4
[INPUT] num. atoms = 2
[INPUT] num. electrons = 2
[INPUT] charge = 0
[INPUT] spin (= nelec alpha-beta = 2S) = 0
[INPUT] symmetry False subgroup None
[INPUT] Mole.unit = angstrom
[INPUT] Symbol           X                Y                Z      unit          X                Y                Z       unit  Magmom
[INPUT]  1 H      0.000000000000   0.000000000000   0.371395000000 AA    0.000000000000   0.000000000000   0.701834834033 Bohr   0.0
[INPUT]  2 H      0.000000000000   0.000000000000  -0.371395000000 AA    0.000000000000   0.0000000000

Initialize <pyscfad.gto.mole.Mole object at 0x7f5c7c7eea70> in <pyscfad.dft.rks.RKS object at 0x7f5c7c7ef640>


init E= -0.96274472276444
LINEAR MIXING SPECIFIED, ALPHA =  0.3
LINEAR MIXING SPECIFIED -- Using alternate order for matrix calculations
  HOMO = -0.426128445925796  LUMO = 0.0151459597741308
cycle= 1 E= -1.2392768795632  delta_E= -0.277  |g|= 4.62e-15  |ddm|= 0.466
LINEAR MIXING SPECIFIED, ALPHA =  0.3
LINEAR MIXING SPECIFIED -- Using alternate order for matrix calculations


/home/awills/anaconda3/envs/pyscfad/lib/python3.10/site-packages/pyscf/gto/mole.py:1293: UserWarning: Function mol.dumps drops attribute coords because it is not JSON-serializable
  warnings.warn(msg)
/home/awills/anaconda3/envs/pyscfad/lib/python3.10/site-packages/pyscf/gto/mole.py:1293: UserWarning: Function mol.dumps drops attribute exp because it is not JSON-serializable
  warnings.warn(msg)
/home/awills/anaconda3/envs/pyscfad/lib/python3.10/site-packages/pyscf/gto/mole.py:1293: UserWarning: Function mol.dumps drops attribute ctr_coeff because it is not JSON-serializable
  warnings.warn(msg)
/home/awills/anaconda3/envs/pyscfad/lib/python3.10/site-packages/pyscf/gto/mole.py:1293: UserWarning: Function mol.dumps drops attribute _enuc because it is not JSON-serializable
  warnings.warn(msg)


  HOMO = -0.389849011442173  LUMO = 0.0274400575140182
cycle= 2 E= -1.18249030867976  delta_E= 0.0568  |g|= 1.47e-14  |ddm|= 0.178
LINEAR MIXING SPECIFIED, ALPHA =  0.3
LINEAR MIXING SPECIFIED -- Using alternate order for matrix calculations
  HOMO = -0.38497907633069  LUMO = 0.0290799291551159
cycle= 3 E= -1.17393479523424  delta_E= 0.00856  |g|= 4.87e-15  |ddm|= 0.108
LINEAR MIXING SPECIFIED, ALPHA =  0.3
LINEAR MIXING SPECIFIED -- Using alternate order for matrix calculations
  HOMO = -0.383069137270172  LUMO = 0.0297201898268427
cycle= 4 E= -1.17043912620963  delta_E= 0.0035  |g|= 3.82e-15  |ddm|= 0.0729
LINEAR MIXING SPECIFIED, ALPHA =  0.3
LINEAR MIXING SPECIFIED -- Using alternate order for matrix calculations
  HOMO = -0.382082357126969  LUMO = 0.0300483994201873
cycle= 5 E= -1.16857112748194  delta_E= 0.00187  |g|= 4.29e-15  |ddm|= 0.0504
LINEAR MIXING SPECIFIED, ALPHA =  0.3
LINEAR MIXING SPECIFIED -- Using alternate order for matrix calculations
  HOMO = -0.381530194856331  

Array(-1.16587383, dtype=float64)

In [7]:
syms = h2.get_chemical_symbols()
pos = h2.positions
atstr = '; '.join(['{} {} {} {}'.format(syms[i], pos[i][0], pos[i][1], pos[i][2]) for i in range(len(syms))])
mol = gto.Mole(atom=atstr, basis='def2-tzvpd')
mol.verbose = 4
mf = dft.RKS(mol)
mf.xc = 'pbe,pbe'
mf.max_cycle = 25
mf.diis_start_cycle = 50
mf.grids.level = 5
mf.kernel()

System: uname_result(system='Linux', node='aegis', release='5.15.0-139-generic', version='#149~20.04.1-Ubuntu SMP Wed Apr 16 08:29:56 UTC 2025', machine='x86_64')  Threads 20
Python 3.10.14 (main, Mar 21 2024, 16:24:04) [GCC 11.2.0]
numpy 1.26.4  scipy 1.11.4  h5py 3.11.0
Date: Mon Jul  7 10:22:42 2025
PySCF version 2.9.0
PySCF path  /home/awills/anaconda3/envs/pyscfad/lib/python3.10/site-packages/pyscf

[CONFIG] conf_file None
[INPUT] verbose = 4
[INPUT] num. atoms = 2
[INPUT] num. electrons = 2
[INPUT] charge = 0
[INPUT] spin (= nelec alpha-beta = 2S) = 0
[INPUT] symmetry False subgroup None
[INPUT] Mole.unit = angstrom
[INPUT] Symbol           X                Y                Z      unit          X                Y                Z       unit  Magmom
[INPUT]  1 H      0.000000000000   0.000000000000   0.371395000000 AA    0.000000000000   0.000000000000   0.701834834033 Bohr   0.0
[INPUT]  2 H      0.000000000000   0.000000000000  -0.371395000000 AA    0.000000000000   0.0000000000

Initialize <pyscfad.gto.mole.Mole object at 0x7f5c7c7ee710> in <pyscfad.dft.rks.RKS object at 0x7f5c7c7eefe0>


init E= -0.96274472276444
  HOMO = -0.426128445925795  LUMO = 0.0151459597741305
cycle= 1 E= -1.16449315048524  delta_E= -0.202  |g|= 0.0568  |ddm|= 0.466
  HOMO = -0.365016504092711  LUMO = 0.0364045808609649
cycle= 2 E= -1.16570674158481  delta_E= -0.00121  |g|= 0.0194  |ddm|= 0.0661
  HOMO = -0.386428280519907  LUMO = 0.0283460765480107


/home/awills/anaconda3/envs/pyscfad/lib/python3.10/site-packages/pyscf/gto/mole.py:1293: UserWarning: Function mol.dumps drops attribute coords because it is not JSON-serializable
  warnings.warn(msg)
/home/awills/anaconda3/envs/pyscfad/lib/python3.10/site-packages/pyscf/gto/mole.py:1293: UserWarning: Function mol.dumps drops attribute exp because it is not JSON-serializable
  warnings.warn(msg)
/home/awills/anaconda3/envs/pyscfad/lib/python3.10/site-packages/pyscf/gto/mole.py:1293: UserWarning: Function mol.dumps drops attribute ctr_coeff because it is not JSON-serializable
  warnings.warn(msg)
/home/awills/anaconda3/envs/pyscfad/lib/python3.10/site-packages/pyscf/gto/mole.py:1293: UserWarning: Function mol.dumps drops attribute _enuc because it is not JSON-serializable
  warnings.warn(msg)


cycle= 3 E= -1.16585365812045  delta_E= -0.000147  |g|= 0.00673  |ddm|= 0.0233
  HOMO = -0.378891713464481  LUMO = 0.0311759662363151
cycle= 4 E= -1.16587121813695  delta_E= -1.76e-05  |g|= 0.00233  |ddm|= 0.00803
  HOMO = -0.381487834463789  LUMO = 0.0301998242956252
cycle= 5 E= -1.16587332933089  delta_E= -2.11e-06  |g|= 0.000808  |ddm|= 0.00278
  HOMO = -0.380586510637871  LUMO = 0.0305385119448808
cycle= 6 E= -1.16587358266024  delta_E= -2.53e-07  |g|= 0.00028  |ddm|= 0.000965
  HOMO = -0.380898593498897  LUMO = 0.0304212155394291
cycle= 7 E= -1.16587361307895  delta_E= -3.04e-08  |g|= 9.7e-05  |ddm|= 0.000334
  HOMO = -0.380790434339829  LUMO = 0.0304618639995392
cycle= 8 E= -1.16587361673063  delta_E= -3.65e-09  |g|= 3.36e-05  |ddm|= 0.000116
  HOMO = -0.380827907196989  LUMO = 0.0304477805389551
cycle= 9 E= -1.16587361716905  delta_E= -4.38e-10  |g|= 1.16e-05  |ddm|= 4.01e-05
  HOMO = -0.380814922889728  LUMO = 0.0304526603979561
Extra cycle  E= -1.16587361722168  delta_E= -5.26

Array(-1.16587362, dtype=float64)

In [8]:
#reference energies:
#[H_TE, O_TE, H2O_AE] <- step 1
kjMol_to_H = 2625.5
refs1 = [-0.5, -75.0673, -974.94/kjMol_to_H]
#[Cl_TE, Cl2_AE, HF+F -> H+F2 barrier]
refs2 = [-460.148, -0.09454200500963746, 0.16920860069537955]
refs3 = refs1 + refs2

mol_params = {'H': {'atoms':['H'], 'coords': [[0,0,0]], 'spin':1, 'charge':0},
              'O': {'atoms':['O'], 'coords':[[0,0,0]], 'spin':2, 'charge':0},
              'H2O': {'atoms':['O','H','H'], 'coords':[[0,0,0],[0,-0.757,0.587],[0,0.757,0.587]], 'spin':0,'charge':0},
              'Cl':{'atoms':['Cl'], 'coords':[[0,0,0]], 'spin':1, 'charge':0},
              'Cl2':{'atoms':['Cl','Cl'], 'coords': [ [0,0,1.008241], [0,0,-1.008241] ], 'spin':0, 'charge':0},
              'HF2':{'atoms':['H', 'F', 'F'], 'coords':[ [0,0,-2.2312757],[0,0,-0.61621628],[0,0,0.8641358] ], 'spin':1, 'charge':0},
              'FH':{'atoms':['F', 'H'], 'coords':[ [0,0,0.09153813], [0,0,-0.82384424] ], 'spin':0, 'charge':0},
              'F':{'atoms':['F'], 'coords':[ [0,0,0] ], 'spin':1, 'charge':0}
             }
mol_dct = {k:0 for k in mol_params.keys()}
for sys in mol_dct.keys():
    atstr = ''
    for idx, at in enumerate(mol_params[sys]['atoms']):
        atstr += '{} {} {} {}\n'.format(at, *mol_params[sys]['coords'][idx])
    mol = gto.Mole(atom = atstr, charge = mol_params[sys]['charge'], spin = mol_params[sys]['spin'])
    mol.basis = 'dzvp'
    mol.build()
    mol.max_memory = 32000

    mol_dct[sys] = mol

mols1 = [mol_dct['H'], mol_dct['O'], mol_dct['H2O']]
mols2 = [mol_dct['Cl'], mol_dct['Cl2'], mol_dct['HF2'], mol_dct['FH'], mol_dct['F']]
mols3 = mols1+mols2

In [9]:
e_lin = []
for mol in mols1:
    print(20*'=')
    print(20*'-')
    print(mol.atom)
    mol.verbose = 4
    print(mol.spin)
    if mol.spin:
        mf = dft.UKS(mol)
    else:
        mf = dft.RKS(mol)
    mf.xc = 'pbe,pbe'
    mf.conv_tol = 1e-5
    # mf.max_cycle = 25
    mf.diis_start_cycle = 9999999999
    mf.grids.level = 4
    mf.kernel(linear_mixing_alpha=0.3)
    e_lin.append(mf.e_tot)
    print(20*'=')

--------------------
H 0 0 0

1
XC library pyscf.dft.libxc version 7.0.0
    S. Lehtola, C. Steigemann, M. J.T. Oliveira, and M. A.L. Marques.,  SoftwareX 7, 1–5 (2018)
XC functionals = pbe,pbe
    J. P. Perdew, K. Burke, and M. Ernzerhof.,  Phys. Rev. Lett. 77, 3865 (1996)
    J. P. Perdew, K. Burke, and M. Ernzerhof.,  Phys. Rev. Lett. 78, 1396 (1997)
    J. P. Perdew, K. Burke, and M. Ernzerhof.,  Phys. Rev. Lett. 77, 3865 (1996)
    J. P. Perdew, K. Burke, and M. Ernzerhof.,  Phys. Rev. Lett. 78, 1396 (1997)
radial grids: 
    Treutler-Ahlrichs [JCP 102, 346 (1995); DOI:10.1063/1.469408] (M4) radial grids
    
becke partition: Becke, JCP 88, 2547 (1988); DOI:10.1063/1.454033
pruning grids: <function nwchem_prune at 0x7f5c80d6fd90>
grids dens level: 4
symmetrized grids: False
atomic radii adjust function: <function treutler_atomic_radii_adjust at 0x7f5c80d6e4d0>
small_rho_cutoff = 1e-07
Set gradient conv threshold to 0.00316228
tot grids = 15712
init E= -0.444008054813723
LINEAR MIX

/home/awills/anaconda3/envs/pyscfad/lib/python3.10/site-packages/pyscf/gto/mole.py:1293: UserWarning: Function mol.dumps drops attribute coords because it is not JSON-serializable
  warnings.warn(msg)
/home/awills/anaconda3/envs/pyscfad/lib/python3.10/site-packages/pyscf/gto/mole.py:1293: UserWarning: Function mol.dumps drops attribute exp because it is not JSON-serializable
  warnings.warn(msg)
/home/awills/anaconda3/envs/pyscfad/lib/python3.10/site-packages/pyscf/gto/mole.py:1293: UserWarning: Function mol.dumps drops attribute ctr_coeff because it is not JSON-serializable
  warnings.warn(msg)


cycle= 1 E= -0.44859376505469  delta_E= -0.00459  |g|= 3.12e-17  |ddm|= 0.472
LINEAR MIXING SPECIFIED, ALPHA =  0.3
LINEAR MIXING SPECIFIED -- Using alternate order for matrix calculations
cycle= 2 E= -0.467101539675951  delta_E= -0.0185  |g|= 1.55e-17  |ddm|= 0.173
LINEAR MIXING SPECIFIED, ALPHA =  0.3
LINEAR MIXING SPECIFIED -- Using alternate order for matrix calculations
cycle= 3 E= -0.476058400462117  delta_E= -0.00896  |g|= 1.41e-18  |ddm|= 0.104
LINEAR MIXING SPECIFIED, ALPHA =  0.3
LINEAR MIXING SPECIFIED -- Using alternate order for matrix calculations
cycle= 4 E= -0.481654671791573  delta_E= -0.0056  |g|= 7.76e-17  |ddm|= 0.0698
LINEAR MIXING SPECIFIED, ALPHA =  0.3
LINEAR MIXING SPECIFIED -- Using alternate order for matrix calculations
cycle= 5 E= -0.485682182176589  delta_E= -0.00403  |g|= 4.98e-17  |ddm|= 0.0488
LINEAR MIXING SPECIFIED, ALPHA =  0.3
LINEAR MIXING SPECIFIED -- Using alternate order for matrix calculations
cycle= 6 E= -0.488702964829824  delta_E= -0.00302  

/home/awills/anaconda3/envs/pyscfad/lib/python3.10/site-packages/pyscf/gto/mole.py:1293: UserWarning: Function mol.dumps drops attribute coords because it is not JSON-serializable
  warnings.warn(msg)
/home/awills/anaconda3/envs/pyscfad/lib/python3.10/site-packages/pyscf/gto/mole.py:1293: UserWarning: Function mol.dumps drops attribute exp because it is not JSON-serializable
  warnings.warn(msg)
/home/awills/anaconda3/envs/pyscfad/lib/python3.10/site-packages/pyscf/gto/mole.py:1293: UserWarning: Function mol.dumps drops attribute ctr_coeff because it is not JSON-serializable
  warnings.warn(msg)


cycle= 1 E= -74.8339778433181  delta_E= 0.0861  |g|= 2.76e-15  |ddm|= 0.854
LINEAR MIXING SPECIFIED, ALPHA =  0.3
LINEAR MIXING SPECIFIED -- Using alternate order for matrix calculations
  alpha nocc = 5  HOMO = -0.328510094610952  LUMO = 0.46584370064746
  beta  nocc = 3  HOMO = -0.300694359031708  LUMO = -0.288231850071901
cycle= 2 E= -75.0777125885972  delta_E= -0.244  |g|= 1.9e-15  |ddm|= 0.353
LINEAR MIXING SPECIFIED, ALPHA =  0.3
LINEAR MIXING SPECIFIED -- Using alternate order for matrix calculations
  alpha nocc = 5  HOMO = -0.323092782687585  LUMO = 0.462932153109518
  beta  nocc = 3  HOMO = -0.288072524875821  LUMO = -0.268501097961729
cycle= 3 E= -75.0163702130113  delta_E= 0.0613  |g|= 1.76e-15  |ddm|= 0.217
LINEAR MIXING SPECIFIED, ALPHA =  0.3
LINEAR MIXING SPECIFIED -- Using alternate order for matrix calculations
  alpha nocc = 5  HOMO = -0.321181436710803  LUMO = 0.461097128093315
  beta  nocc = 3  HOMO = -0.282458181960745  LUMO = -0.258350766467232
cycle= 4 E= -75.00

/home/awills/anaconda3/envs/pyscfad/lib/python3.10/site-packages/pyscf/gto/mole.py:1293: UserWarning: Function mol.dumps drops attribute coords because it is not JSON-serializable
  warnings.warn(msg)
/home/awills/anaconda3/envs/pyscfad/lib/python3.10/site-packages/pyscf/gto/mole.py:1293: UserWarning: Function mol.dumps drops attribute exp because it is not JSON-serializable
  warnings.warn(msg)
/home/awills/anaconda3/envs/pyscfad/lib/python3.10/site-packages/pyscf/gto/mole.py:1293: UserWarning: Function mol.dumps drops attribute ctr_coeff because it is not JSON-serializable
  warnings.warn(msg)


cycle= 1 E= -79.7060675197705  delta_E= -3.4  |g|= 7.51e-15  |ddm|= 1.29
LINEAR MIXING SPECIFIED, ALPHA =  0.3
LINEAR MIXING SPECIFIED -- Using alternate order for matrix calculations
  HOMO = -0.196134743795981  LUMO = 0.0499663010854549
cycle= 2 E= -73.9335085907085  delta_E= 5.77  |g|= 7.32e-15  |ddm|= 0.85
LINEAR MIXING SPECIFIED, ALPHA =  0.3
LINEAR MIXING SPECIFIED -- Using alternate order for matrix calculations
  HOMO = -0.271635370792332  LUMO = 0.0295224387183841
cycle= 3 E= -76.6948833495778  delta_E= -2.76  |g|= 4.46e-15  |ddm|= 0.342
LINEAR MIXING SPECIFIED, ALPHA =  0.3
LINEAR MIXING SPECIFIED -- Using alternate order for matrix calculations
  HOMO = -0.259186407919165  LUMO = 0.0330252690996675
cycle= 4 E= -76.3904740538518  delta_E= 0.304  |g|= 4.64e-15  |ddm|= 0.24
LINEAR MIXING SPECIFIED, ALPHA =  0.3
LINEAR MIXING SPECIFIED -- Using alternate order for matrix calculations
  HOMO = -0.256534476722749  LUMO = 0.0339792963746845
cycle= 5 E= -76.3667340399303  delta_E= 0

In [10]:
e_def = []
for mol in mols1:
    print(20*'=')
    print(20*'-')
    print(mol.atom)
    mol.verbose = 4
    print(mol.spin)
    if mol.spin:
        mf = dft.UKS(mol)
    else:
        mf = dft.RKS(mol)
    mf.xc = 'pbe,pbe'
    mf.conv_tol = 1e-5
    # mf.max_cycle = 25
    mf.diis_start_cycle = 9999999999
    mf.grids.level = 4
    mf.kernel()
    e_def.append(mf.e_tot)
    print(20*'=')

--------------------
H 0 0 0

1
XC library pyscf.dft.libxc version 7.0.0
    S. Lehtola, C. Steigemann, M. J.T. Oliveira, and M. A.L. Marques.,  SoftwareX 7, 1–5 (2018)
XC functionals = pbe,pbe
    J. P. Perdew, K. Burke, and M. Ernzerhof.,  Phys. Rev. Lett. 77, 3865 (1996)
    J. P. Perdew, K. Burke, and M. Ernzerhof.,  Phys. Rev. Lett. 78, 1396 (1997)
    J. P. Perdew, K. Burke, and M. Ernzerhof.,  Phys. Rev. Lett. 77, 3865 (1996)
    J. P. Perdew, K. Burke, and M. Ernzerhof.,  Phys. Rev. Lett. 78, 1396 (1997)
radial grids: 
    Treutler-Ahlrichs [JCP 102, 346 (1995); DOI:10.1063/1.469408] (M4) radial grids
    
becke partition: Becke, JCP 88, 2547 (1988); DOI:10.1063/1.454033
pruning grids: <function nwchem_prune at 0x7f5c80d6fd90>
grids dens level: 4
symmetrized grids: False
atomic radii adjust function: <function treutler_atomic_radii_adjust at 0x7f5c80d6e4d0>
small_rho_cutoff = 1e-07
Set gradient conv threshold to 0.00316228
tot grids = 15712
init E= -0.444008054813723
cycle= 1 E

/home/awills/anaconda3/envs/pyscfad/lib/python3.10/site-packages/pyscf/gto/mole.py:1293: UserWarning: Function mol.dumps drops attribute coords because it is not JSON-serializable
  warnings.warn(msg)
/home/awills/anaconda3/envs/pyscfad/lib/python3.10/site-packages/pyscf/gto/mole.py:1293: UserWarning: Function mol.dumps drops attribute exp because it is not JSON-serializable
  warnings.warn(msg)
/home/awills/anaconda3/envs/pyscfad/lib/python3.10/site-packages/pyscf/gto/mole.py:1293: UserWarning: Function mol.dumps drops attribute ctr_coeff because it is not JSON-serializable
  warnings.warn(msg)


Extra cycle  E= -0.49768791496599  delta_E= -4.29e-08  |g|= 1.51e-05  |ddm|= 0.000448
converged SCF energy = -0.49768791496599  <S^2> = 0.75  2S+1 = 2
--------------------
O 0 0 0

2
XC library pyscf.dft.libxc version 7.0.0
    S. Lehtola, C. Steigemann, M. J.T. Oliveira, and M. A.L. Marques.,  SoftwareX 7, 1–5 (2018)
XC functionals = pbe,pbe
    J. P. Perdew, K. Burke, and M. Ernzerhof.,  Phys. Rev. Lett. 77, 3865 (1996)
    J. P. Perdew, K. Burke, and M. Ernzerhof.,  Phys. Rev. Lett. 78, 1396 (1997)
    J. P. Perdew, K. Burke, and M. Ernzerhof.,  Phys. Rev. Lett. 77, 3865 (1996)
    J. P. Perdew, K. Burke, and M. Ernzerhof.,  Phys. Rev. Lett. 78, 1396 (1997)
radial grids: 
    Treutler-Ahlrichs [JCP 102, 346 (1995); DOI:10.1063/1.469408] (M4) radial grids
    
becke partition: Becke, JCP 88, 2547 (1988); DOI:10.1063/1.454033
pruning grids: <function nwchem_prune at 0x7f5c80d6fd90>
grids dens level: 4
symmetrized grids: False
atomic radii adjust function: <function treutler_atomic_rad

/home/awills/anaconda3/envs/pyscfad/lib/python3.10/site-packages/pyscf/gto/mole.py:1293: UserWarning: Function mol.dumps drops attribute coords because it is not JSON-serializable
  warnings.warn(msg)
/home/awills/anaconda3/envs/pyscfad/lib/python3.10/site-packages/pyscf/gto/mole.py:1293: UserWarning: Function mol.dumps drops attribute exp because it is not JSON-serializable
  warnings.warn(msg)
/home/awills/anaconda3/envs/pyscfad/lib/python3.10/site-packages/pyscf/gto/mole.py:1293: UserWarning: Function mol.dumps drops attribute ctr_coeff because it is not JSON-serializable
  warnings.warn(msg)


cycle= 4 E= -74.994621724201  delta_E= 4.82e-05  |g|= 0.0329  |ddm|= 0.0588
  alpha nocc = 5  HOMO = -0.30398263961015  LUMO = 0.464795052292753
  beta  nocc = 3  HOMO = -0.257491124562418  LUMO = -0.221434834306098
cycle= 5 E= -74.9945511836023  delta_E= 7.05e-05  |g|= 0.0352  |ddm|= 0.0622
  alpha nocc = 5  HOMO = -0.33337197187718  LUMO = 0.449427710448286
  beta  nocc = 3  HOMO = -0.286365497735567  LUMO = -0.249371307260116
cycle= 6 E= -74.9944908162653  delta_E= 6.04e-05  |g|= 0.0367  |ddm|= 0.0657
  alpha nocc = 5  HOMO = -0.302349904318724  LUMO = 0.465675918669328
  beta  nocc = 3  HOMO = -0.255868910435955  LUMO = -0.219870683570045
cycle= 7 E= -74.9944015789096  delta_E= 8.92e-05  |g|= 0.0393  |ddm|= 0.0695
  alpha nocc = 5  HOMO = -0.335183676713033  LUMO = 0.448503151698873
  beta  nocc = 3  HOMO = -0.288145085829398  LUMO = -0.251097244003402
cycle= 8 E= -74.9943276611061  delta_E= 7.39e-05  |g|= 0.041  |ddm|= 0.0734
  alpha nocc = 5  HOMO = -0.300523409175109  LUMO = 0.4

/home/awills/anaconda3/envs/pyscfad/lib/python3.10/site-packages/pyscf/gto/mole.py:1293: UserWarning: Function mol.dumps drops attribute coords because it is not JSON-serializable
  warnings.warn(msg)
/home/awills/anaconda3/envs/pyscfad/lib/python3.10/site-packages/pyscf/gto/mole.py:1293: UserWarning: Function mol.dumps drops attribute exp because it is not JSON-serializable
  warnings.warn(msg)
/home/awills/anaconda3/envs/pyscfad/lib/python3.10/site-packages/pyscf/gto/mole.py:1293: UserWarning: Function mol.dumps drops attribute ctr_coeff because it is not JSON-serializable
  warnings.warn(msg)


cycle= 3 E= -75.5117201953283  delta_E= 0.485  |g|= 1.94  |ddm|=  2.2
  HOMO = 0.163023389907445  LUMO = 0.20425450580736
cycle= 4 E= -73.8896799843902  delta_E= 1.62  |g|= 3.06  |ddm|= 4.14
  HOMO = -1.52460246781813  LUMO = -0.214379452085613
cycle= 5 E= -73.7269986569405  delta_E= 0.163  |g|= 2.93  |ddm|= 4.96
  HOMO = 0.325102050087472  LUMO = 0.337546679108964
cycle= 6 E= -71.4092675593703  delta_E= 2.32  |g|= 3.67  |ddm|= 9.53
  HOMO = -2.09362986551626  LUMO = -0.346547142356095
cycle= 7 E= -73.0366997587514  delta_E= -1.63  |g|= 3.08  |ddm|=  9.7
  HOMO = 0.353415779210066  LUMO = 0.378227461540496
cycle= 8 E= -70.4848162109511  delta_E= 2.55  |g|= 3.74  |ddm|= 12.4
  HOMO = -2.2581451680575  LUMO = -0.40638379119597
cycle= 9 E= -72.866458098836  delta_E= -2.38  |g|= 3.11  |ddm|= 12.4
  HOMO = 0.358556954625864  LUMO = 0.390101153094835
cycle= 10 E= -70.3363326181776  delta_E= 2.53  |g|= 3.75  |ddm|= 12.7
  HOMO = -2.28283614874907  LUMO = -0.416247702056764
cycle= 11 E= -72.84

In [11]:
for idx, i in enumerate(e_lin):
    print(10*'-')
    print(mols3[idx].atom)
    print(abs(e_def[idx]-e_lin[idx]))
    print(10*'-')

----------
H 0 0 0

2.396200460275111e-10
----------
----------
O 0 0 0

0.060330209726018325
----------
----------
O 0 0 0
H 0 -0.757 0.587
H 0 0.757 0.587

6.0353130259613295
----------


We can see that the linear mixing scheme leads to converged results for all H, O, and H2O molecules, whereas the default DIIS scheme does not converge within 50 steps for O and H2O.

In [22]:
syms = h2.get_chemical_symbols()
pos = h2.positions
atstr = '; '.join(['{} {} {} {}'.format(syms[i], pos[i][0], pos[i][1], pos[i][2]) for i in range(len(syms))])
mol = gto.Mole(atom=atstr, basis='def2-tzvpd')
mol.verbose = 4
mf = dft.RKS(mol)
mf.xc = 'pbe,pbe'
mf.max_cycle = -1
mf.kernel()
mf.max_cycle=25
mf.diis_start_cycle = 50
mf.grids.level = 5
deriv_func = partial(mf.kernel, linear_mixing_alpha=0.3)
g = jax.grad(deriv_func)(mf.make_rdm1())

System: uname_result(system='Linux', node='aegis', release='5.15.0-139-generic', version='#149~20.04.1-Ubuntu SMP Wed Apr 16 08:29:56 UTC 2025', machine='x86_64')  Threads 20
Python 3.10.14 (main, Mar 21 2024, 16:24:04) [GCC 11.2.0]
numpy 1.26.4  scipy 1.11.4  h5py 3.11.0
Date: Mon Jul  7 10:34:45 2025
PySCF version 2.9.0
PySCF path  /home/awills/anaconda3/envs/pyscfad/lib/python3.10/site-packages/pyscf

[CONFIG] conf_file None
[INPUT] verbose = 4
[INPUT] num. atoms = 2
[INPUT] num. electrons = 2
[INPUT] charge = 0
[INPUT] spin (= nelec alpha-beta = 2S) = 0
[INPUT] symmetry False subgroup None
[INPUT] Mole.unit = angstrom
[INPUT] Symbol           X                Y                Z      unit          X                Y                Z       unit  Magmom
[INPUT]  1 H      0.000000000000   0.000000000000   0.371395000000 AA    0.000000000000   0.000000000000   0.701834834033 Bohr   0.0
[INPUT]  2 H      0.000000000000   0.000000000000  -0.371395000000 AA    0.000000000000   0.0000000000

Initialize <pyscfad.gto.mole.Mole object at 0x7f5b9869c970> in <pyscfad.dft.rks.RKS object at 0x7f5b9869ea40>


init E= -0.962744720439682
  HOMO = -0.426128444528908  LUMO = 0.0151459599272171
SCF not converged.
SCF energy = -0.962744720439682
XC library pyscf.dft.libxc version 7.0.0
    S. Lehtola, C. Steigemann, M. J.T. Oliveira, and M. A.L. Marques.,  SoftwareX 7, 1–5 (2018)
XC functionals = pbe,pbe
    J. P. Perdew, K. Burke, and M. Ernzerhof.,  Phys. Rev. Lett. 77, 3865 (1996)
    J. P. Perdew, K. Burke, and M. Ernzerhof.,  Phys. Rev. Lett. 78, 1396 (1997)
    J. P. Perdew, K. Burke, and M. Ernzerhof.,  Phys. Rev. Lett. 77, 3865 (1996)
    J. P. Perdew, K. Burke, and M. Ernzerhof.,  Phys. Rev. Lett. 78, 1396 (1997)
radial grids: 
    Treutler-Ahlrichs [JCP 102, 346 (1995); DOI:10.1063/1.469408] (M4) radial grids
    
becke partition: Becke, JCP 88, 2547 (1988); DOI:10.1063/1.454033
pruning grids: <function nwchem_prune at 0x7f5c80d6fd90>
grids dens level: 5
symmetrized grids: False
atomic radii adjust function: <function treutler_atomic_radii_adjust at 0x7f5c80d6e4d0>
small_rho_cutoff = 1e

/home/awills/anaconda3/envs/pyscfad/lib/python3.10/site-packages/pyscf/gto/mole.py:1293: UserWarning: Function mol.dumps drops attribute coords because it is not JSON-serializable
  warnings.warn(msg)
/home/awills/anaconda3/envs/pyscfad/lib/python3.10/site-packages/pyscf/gto/mole.py:1293: UserWarning: Function mol.dumps drops attribute exp because it is not JSON-serializable
  warnings.warn(msg)
/home/awills/anaconda3/envs/pyscfad/lib/python3.10/site-packages/pyscf/gto/mole.py:1293: UserWarning: Function mol.dumps drops attribute ctr_coeff because it is not JSON-serializable
  warnings.warn(msg)
/home/awills/anaconda3/envs/pyscfad/lib/python3.10/site-packages/pyscf/gto/mole.py:1293: UserWarning: Function mol.dumps drops attribute _enuc because it is not JSON-serializable
  warnings.warn(msg)


cycle= 1 E= -1.14041895401595  delta_E= 0.0241  |g|= 6.55e-15  |ddm|= 0.0661
LINEAR MIXING SPECIFIED, ALPHA =  0.3
LINEAR MIXING SPECIFIED -- Using alternate order for matrix calculations
  HOMO = -0.377867792352748  LUMO = 0.0315008813776309
cycle= 2 E= -1.161235539905  delta_E= -0.0208  |g|= 9.13e-15  |ddm|= 0.0124
LINEAR MIXING SPECIFIED, ALPHA =  0.3
LINEAR MIXING SPECIFIED -- Using alternate order for matrix calculations
  HOMO = -0.379423343737821  LUMO = 0.0309375145276563
cycle= 3 E= -1.1636926511177  delta_E= -0.00246  |g|= 3.7e-15  |ddm|= 0.0059
LINEAR MIXING SPECIFIED, ALPHA =  0.3
LINEAR MIXING SPECIFIED -- Using alternate order for matrix calculations
  HOMO = -0.380040135481306  LUMO = 0.0307176386143726
cycle= 4 E= -1.16466159335873  delta_E= -0.000969  |g|= 5.71e-15  |ddm|= 0.00331
LINEAR MIXING SPECIFIED, ALPHA =  0.3
LINEAR MIXING SPECIFIED -- Using alternate order for matrix calculations
  HOMO = -0.380364493721904  LUMO = 0.0306035458650712
cycle= 5 E= -1.1651693765

So we are able to access derivatives of the kernel function w.r.t. the input density matrix, which is what we want to show.

In [23]:
g

Array([[ 3.44906958e-05,  2.87965579e-05,  1.57970608e-05,
         1.75294756e-21, -4.46173910e-21,  1.83580716e-06,
         2.65097789e-22, -1.60076999e-22,  3.43522245e-07,
         9.36566147e-06,  1.69272074e-05,  1.29763814e-05,
         1.16482956e-22, -7.04627303e-22,  2.04007260e-05,
         3.98538329e-22, -2.45882614e-22,  1.03446367e-05],
       [ 2.51173718e-05,  4.03192744e-05,  3.32483241e-05,
         1.76450904e-21,  3.71793408e-21,  2.42714101e-07,
         7.66567090e-22, -4.85548197e-22,  1.40333728e-07,
         1.51032045e-05,  2.93751788e-05,  2.84982907e-05,
         1.61059467e-21, -5.30513652e-22,  1.49739922e-05,
        -1.09345941e-22, -1.27522854e-21,  1.86419604e-05],
       [ 1.31810332e-05,  2.95149014e-05,  3.62595815e-05,
        -1.85520987e-21,  9.23941969e-22, -6.35514243e-07,
         1.29195483e-21, -4.44344589e-22, -2.14410277e-06,
         1.09829261e-05,  2.56847080e-05,  3.32916006e-05,
        -5.30893912e-22, -1.76337802e-21,  4.78057361e